# Purpose

To answer the following question:
* Which LA neighbourhood has the most listings?
* What is the most popular neighbourhood, according to reviews?
* What is the best time of year to visit LA?
* During which period of the year is the booking price cheapest?
* Can we predict the price of a new listing?
* Can we predict the price of an existing listing in the future? Let's say listing#109?

# Setup


## Library import
We import all the required Python libraries

In [0]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
# from IPython.core.display import display, HTML
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None
# display(HTML("<style>.container { width:85% !important; }</style>"))
# pd.options.display.float_format =   {:,}'.format

# Visualizations
import plotly.express as px
import plotly.graph_objects as go

# ML
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.model_selection import learning_curve, GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix

# Others
from tqdm.notebook import tqdm


## Helper functions



In [0]:
# Define helper functions for eda and data cleansing

'''This function changes data columns to the proper format'''
def clean_date(date_col):
  return date_col.astype(np.datetime64)

'''This function changes price columns to the proper format'''
def clean_price(price_col):
  if price_col.dtype == 'float64':
    return price_col
  else:
    return price_col.str.replace(r'[^0-9.]','',regex=True).astype('float64')

'''This function changes 't'/'f' columns to the proper format'''
def clean_boolean(boolean_col):
  return boolean_col.replace('t',1).replace('f',0)

In [0]:
# Define helper functions for model optimisation

'''This function picks the best parameters for a model from a parameters dict, using n_fold cross validation'''
def cv_optimize(model, parameters, X, y, n_jobs=1, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(model, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func)
    else:
        gs = GridSearchCV(model, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X, y)
    # print("BEST", gs.best_params_, gs.best_score_, gs.cv_results_) # Only use if you want to see the best parameters
    best = gs.best_estimator_
    return best

# Data import
We retrieve all the required data for the analysis.

In [0]:
# set up google drive data for google colab. To ignore if you are working locally

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# amend your path here
path = '/content/drive/My Drive/Colab Notebooks/ML_01_Los Angeles travel exploration/Data'

neighbourhoods = pd.read_csv(path + '/neighbourhoods.csv')
listings = pd.read_csv(path + '/listings.csv')
detailed_listings = pd.read_csv(path + '/detailed_listings.csv')
reviews = pd.read_csv(path + '/reviews.csv')
calendar = pd.read_csv(path + '/calendar.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (61,62) have mixed types.Specify dtype option on import or set low_memory=False.



#EDA

In [0]:
neighbourhoods.describe(include='all')

,neighbourhood_group,neighbourhood
count,270,270
unique,3,270
top,City of Los Angeles,Leimert Park
freq,114,1


In [0]:
listings.describe(include='all')

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
count,3.704800e+04,37046,3.704800e+04,37043,37048,37048,37048.000000,37048.000000,37048,37048.000000,37048.000000,37048.000000,29413,29413.000000,37048.000000,37048.000000
unique,NaN,36324,NaN,7924,3,264,NaN,NaN,4,NaN,NaN,NaN,1693,NaN,NaN,NaN
top,NaN,"Artist Community for Creatives, Month to Month!",NaN,David,City of Los Angeles,Hollywood,NaN,NaN,Entire home/apt,NaN,NaN,NaN,2020-03-15,NaN,NaN,NaN
freq,NaN,21,NaN,349,21579,1876,NaN,NaN,23305,NaN,NaN,NaN,880,NaN,NaN,NaN
mean,2.519295e+07,NaN,9.254123e+07,NaN,NaN,NaN,34.050421,-118.318055,NaN,227.916379,12.715369,35.201387,NaN,1.605461,8.125513,168.061353
std,1.278064e+07,NaN,9.537659e+07,NaN,NaN,NaN,0.121134,0.165569,NaN,685.160627,26.759290,64.277015,NaN,1.750648,23.030745,142.799526
min,1.090000e+02,NaN,5.210000e+02,NaN,NaN,NaN,33.338830,-118.934220,NaN,0.000000,1.000000,0.000000,NaN,0.010000,1.000000,0.000000
25%,1.518486e+07,NaN,1.414556e+07,NaN,NaN,NaN,33.998350,-118.422363,NaN,69.000000,1.000000,1.000000,NaN,0.300000,1.000000,5.000000
50%,2.629570e+07,NaN,5.115448e+07,NaN,NaN,NaN,34.061465,-118.342670,NaN,109.000000,3.000000,8.000000,NaN,0.960000,2.000000,153.000000
75%,3.684847e+07,NaN,1.530491e+08,NaN,NaN,NaN,34.104190,-118.250908,NaN,185.000000,30.000000,40.000000,NaN,2.410000,5.000000,336.000000


In [0]:
detailed_listings.describe(include='all')

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,...,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,3.704800e+04,37048,3.704800e+04,37048,37046,34928,27294,35576,37048,24525,18169,22468,21002,23246,25331,0.0,0.0,37048,0.0,3.704800e+04,37048,37043,37043,36945,23681,27937,27937,31024,37043,37043,37043,30430,37043.000000,37043.000000,37048,37043,37043,37048,34952,37048,...,37048,37048.000000,37048.000000,37048.000000,37048.000000,37048.000000,3.704800e+04,37048.000000,3.704800e+04,37048,37048,37048.000000,37048.000000,37048.000000,37048.000000,37048,37048.000000,37048.000000,29413,29413,28962.000000,28914.000000,28915.000000,28902.000000,28913.000000,28898.000000,28894.000000,37048,9296,25431,37048,37048,37048,37048,37048,37048.000000,37048.000000,37048.000000,37048.000000,29413.000000
unique,NaN,37048,NaN,3,36324,31928,24650,33834,1,20574,15072,18740,17951,18956,21034,NaN,NaN,35918,NaN,NaN,21774,7924,3579,913,12939,4,73,99,2,21695,21695,452,NaN,NaN,490,2,2,432,164,264,...,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92,1,NaN,NaN,NaN,NaN,3,NaN,NaN,2871,1693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,5448,11,2,1,9,2,2,NaN,NaN,NaN,NaN,NaN
top,NaN,https://www.airbnb.com/rooms/27513639,NaN,2020-05-09,"Artist Community for Creatives, Month to Month!","Follow, Like, Share @HadrianHome We are offeri...","At Zeus, we focus exclusively on 30 day+ stays...",Imagine a nursing home right next to Beverly h...,none,"For more than a century, Los Angeles has lured...",This Blueground apartment can be booked for a ...,Uber and Lyft are commonly used as they are re...,The apartment is completely private and you ha...,The entire apartment is yours to enjoy! You’ll...,- A passport or ID number and an address have ...,NaN,NaN,https://a0.muscache.com/im/pictures/64e9290f-6...,NaN,NaN,https://www.airbnb.com/users/show/48005494,David,2015-11-02,"Los Angeles, California, United States",Zeus is reinventing corporate housing. We do t...,within an hour,100%,100%,f,https://a0.muscache.com/im/pictures/user/7776f...,https://a0.muscache.com/im/pictures/user/7776f...,Hollywood,NaN,NaN,"['email', 'phone']",t,f,"Los Angeles, CA, United States",Hollywood,Hollywood,...,$0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 months ago,t,NaN,NaN,NaN,NaN,2020-05-09,NaN,NaN,2019-07-07,2020-03-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,Exempt - This listing is a transient occupancy...,"{""City of Los Angeles"","" CA""}",f,f,strict_14_with_grace_period,f,f,NaN,NaN,NaN,NaN,NaN
freq,NaN,1,NaN,20173,21,59,205,29,37048,98,179,58,48,190,78,NaN,NaN,24,NaN,NaN,206,349,231,18225,206,19314,20777,9563,25021,206,206,1974,NaN,NaN,2897,36930,21440,20144,2989,1876,...,18256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6432,

In [0]:
reviews.describe(include='all')

,listing_id,date
count,1.304141e+06,1304141
unique,NaN,3644
top,NaN,2019-11-11
freq,NaN,2747
mean,1.582831e+07,NaN
std,1.087678e+07,NaN
min,1.090000e+02,NaN
25%,6.121930e+06,NaN
50%,1.531518e+07,NaN
75%,2.323449e+07,NaN


In [0]:
listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,109,Amazing bright elegant condo park front *UPGRA...,521,Paolo,Other Cities,Culver City,33.98209,-118.38494,Entire home/apt,122,30,2,2016-05-15,0.02,1,38
1,344,Family perfect;Pool;Near Studios!,767,Melissa,Other Cities,Burbank,34.16562,-118.33458,Entire home/apt,168,2,8,2019-10-19,0.17,1,97
2,2708,Mirrored Mini-Suite with Fireplace - W. Hollywood,3008,Chas.,City of Los Angeles,Hollywood,34.09768,-118.34602,Private room,79,30,24,2020-03-17,0.33,2,281
3,2732,Zen Life at the Beach,3041,Yoga Priestess,Other Cities,Santa Monica,34.00475,-118.48127,Private room,155,1,21,2019-12-27,0.19,2,365
4,2864,* Beautiful Master Suite/Jacuzzi Tub/*,3207,Bernadine,Other Cities,Bellflower,33.87619,-118.11397,Entire home/apt,80,14,0,NaN,NaN,1,0


In [0]:
detailed_listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,...,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,109,https://www.airbnb.com/rooms/109,20200508043406,2020-05-09,Amazing bright elegant condo park front *UPGRA...,"*** Unit upgraded with new bamboo flooring, br...","*** Unit upgraded with new bamboo flooring, br...","*** Unit upgraded with new bamboo flooring, br...",none,NaN,NaN,NaN,NaN,NaN,Camelot NEW RESIDENTS’ GENERAL INFORMATION F...,NaN,NaN,https://a0.muscache.com/im/pictures/4321499/1d...,NaN,521,https://www.airbnb.com/users/show/521,Paolo,2008-06-27,"San Francisco, California, United States",Search for me on the Internet with the keyword...,within a day,100%,0%,f,https://a0.muscache.com/im/users/521/profile_p...,https://a0.muscache.com/im/users/521/profile_p...,Culver City,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,"Culver City, CA, United States",Culver City,Culver City,...,$25.00,30,730,30,30,730,730,30.0,730.0,3 months ago,t,0,0,0,38,2020-05-09,2,0,2011-08-15,2016-05-15,80.0,10.0,10.0,6.0,8.0,10.0,8.0,f,NaN,"{""Culver City"","" CA""}",f,f,strict_14_with_grace_period,t,f,1,1,0,0,0.02
1,344,https://www.airbnb.com/rooms/344,20200508043406,2020-05-08,Family perfect;Pool;Near Studios!,This home is perfect for families; aspiring ch...,"Cheerful & comfortable; near studios, amusemen...",This home is perfect for families; aspiring ch...,none,Quiet-yet-close to all the fun in LA! Hollywoo...,"One dog may be on premises, friendly and cared...",Short drive to subway and elevated trains runn...,"Pool, patio and self-contained main house all ...",Host and caretaker may be available throughout...,Host asks that guests refrain from partying lo...,NaN,NaN,https://a0.muscache.com/im/pictures/cc4b724d-d...,NaN,767,https://www.airbnb.com/users/show/767,Melissa,2008-07-11,"Burbank, California, United States","Single mother, CEO and Owner of an internation...",within a day,60%,33%,f,https://a0.muscache.com/im/users/767/profile_p...,https://a0.muscache.com/im/users/767/profile_p...,Burbank,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'kba', ...",t,t,"Burbank, CA, United States",Burbank,Burbank,...,$0.00,2,14,2,2,14,14,2.0,14.0,8 months ago,t,0,1,6,97,2020-05-08,8,2,2016-06-14,2019-10-19,97.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,NaN,t,f,flexible,f,f,1,1,0,0,0.17
2,2708,https://www.airbnb.com/rooms/2708,20200508043406,2020-05-09,Mirrored Mini-Suite with Fireplace - W. Hollywood,Our best memory foam pillows you'll ever sleep...,Flickering fireplace. Blendtec® Designer 625 ...,Our best memory foam pillows you'll ever sleep...,none,We are minutes away from the Mentor Language I...,Blendtec® Designer 625 

# Answering questions

## Which LA neighbourhood has the most listings?

In [0]:
nbh = listings.groupby(by='neighbourhood')['id'].agg(['count', lambda x: x.count()/listings.shape[0]])
nbh.columns = ['Listing count', '% of total']
nbh['% of total'] = nbh['% of total'].apply(lambda x:f'{x:.1%}')
nbh.sort_values(by = '% of total', ascending=False, inplace=True)
nbh.head()

,Listing count,% of total
neighbourhood,,
Hollywood,1876,5.1%
Venice,1868,5.0%
Long Beach,1740,4.7%
Santa Monica,1258,3.4%
Downtown,1267,3.4%


## What is the most popular neighbourhood, according to reviews?

In [0]:
nbhrv = detailed_listings.groupby(by='neighbourhood')['number_of_reviews', 'review_scores_rating'].agg(
    {'number_of_reviews': 'sum',
    'review_scores_rating': 'mean'})
nbhrv.columns = ['Review count', 'Average rating']
nbhrv['Average rating'] = nbhrv['Average rating'].apply(lambda x:f'{x:.2f}')
nbhrv.dropna(inplace=True)
nbhrv.replace(["nan"], np.nan, inplace = True)
nbhrv.sort_values(by = 'Average rating', ascending=False, inplace=True)
nbhrv.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Review count,Average rating
neighbourhood,,
Hawaiian Gardens,47,99.00
Porter Ranch,186,98.67
Mission Hills,233,98.67
Bell Gardens,37,98.67
Irwindale,265,98.50


## What is the best time of year to visit LA?

In [0]:
reviews['date'] = clean_date(reviews['date'])

In [0]:
reviews.groupby(by=pd.Grouper(key='date', freq='Y')).count()

,listing_id
date,
2009-12-31,22
2010-12-31,389
2011-12-31,1791
2012-12-31,4275
2013-12-31,10319
2014-12-31,25681
2015-12-31,60238
2016-12-31,120729
2017-12-31,196251


In [0]:
# remove incomplete 1st & last year to ensure all months are treated equal
reviews_trimmed = reviews[(reviews['date'].dt.year>2009)&(reviews['date'].dt.year<2020)]

In [0]:
reviews_trimmed['month']=reviews_trimmed['date'].dt.month
reviews_grp = reviews_trimmed.groupby(by='month')[['listing_id']].count()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
fig = px.bar(reviews_grp,
             x=reviews_grp.index,
             y='listing_id'
             )

fig.update_layout(title="No. of reviews per month",
                  xaxis_title="Month",
                  yaxis_title="No. of reviews",
                  )

fig.show()

## During which period of the year is the booking price cheapest?

In [0]:
calendar['date'] = clean_date(calendar['date'])
calendar['price'] = clean_price(calendar['price'])
calendar_grp = calendar.groupby(by=pd.Grouper(key='date', freq = 'D'))[['listing_id', 'price']] \
  .agg({'listing_id':'count', 'price': 'mean'})

#drop a few first and last rows due to missing data, which can skew the result
calendar_grp = calendar_grp[1:-2]

In [71]:
fig = px.line(calendar_grp,
             x=calendar_grp.index,
             y='price'
             )

fig.update_layout(title="Daily average listing price",
                  xaxis_title="Day",
                  yaxis_title="Average listing price",
                  )

fig.show()


## Can we predict the price of a new listing?

In [0]:
listing_combined = listings.merge(detailed_listings, on='id',
                 how='outer', suffixes=('', '_y'))
listing_combined.drop(listing_combined.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
listing_combined.filter(regex='price').columns.tolist()
listing_combined.drop(['weekly_price', 'monthly_price'], axis=1, inplace=True)

In [0]:
# # Create test/train mask
# itrain, itest = train_test_split(range(listing_combined.shape[0]), train_size=0.6)
# mask=np.ones(listing_combined.shape[0], dtype='int')
# mask[itrain]=1
# mask[itest]=0
# mask = (mask==1)

In [0]:
# Choose relevant columns that a new listing may have to prevent data leakage 
object_cols = ['neighbourhood','room_type', 'property_type']
number_cols = ['bathrooms' ,'bedrooms' ,'beds', 'square_feet','price', 'minimum_nights',
               'availability_365' ,'accommodates','guests_included']
bool_cols = ['requires_license', 'instant_bookable', 'is_business_travel_ready',
             'require_guest_profile_picture', 'require_guest_phone_verification']

In [0]:
# Create training data df
df = listing_combined[object_cols+bool_cols + number_cols]

In [0]:
# Onehotencode categorical data, drop na values
df = df.join(pd.get_dummies(df[object_cols]))
df.drop(object_cols,axis=1,inplace=True)
df.dropna(axis=0,inplace=True)

In [0]:
# Clean boolean columns
df[bool_cols] = clean_boolean(df[bool_cols])

In [0]:
# Define X, y
y = df.pop('price')
X = df

In [0]:
# Scale numerical columns
sc = StandardScaler()
numeric_cols = X.select_dtypes(['int64','float64']).columns

sc_transformed_cols = pd.DataFrame(sc.fit_transform(X[numeric_cols]), columns=numeric_cols, index = X.index)
X.drop(numeric_cols,axis=1,inplace=True)
X = X.merge(sc_transformed_cols,left_index= True, right_index = True, how='left')

In [0]:
# Split train test df
X_train,X_test,y_train,y_test =train_test_split(X,y)

In [0]:
# Define models
names = ['RandomForestRegressor', 'AdaBoostRegressor', 
          'BaggingRegressor', 'GradientBoostingRegressor']
models = [RandomForestRegressor(), AdaBoostRegressor(), 
          BaggingRegressor(), GradientBoostingRegressor()]

In [0]:
# Fit, train, evaluate models 
scores = {}
for name, model in tqdm(list(zip(names,models))): 
    model.fit(X_train, y_train)
    score = model.score(X_test,y_test)
    scores[name] = score
scores

{'AdaBoostRegressor': 0.987676138248658,
 'BaggingRegressor': 0.9837823621947521,
 'GradientBoostingRegressor': 0.9949502126992082,
 'RandomForestRegressor': 0.9885219236777995}

In [0]:
model = GradientBoostingRegressor()
model.fit(X_train, y_train)
Feature_importance = pd.DataFrame(list(zip(X_train.columns, model.feature_importances_)))
Feature_importance.columns = ['Feature', 'Importance']
px.bar(data_frame=Feature_importance.sort_values(by='Importance', ascending=False).head(10),
       x = 'Feature',
       y = 'Importance')

###Parameter tuning

In [0]:
model = GradientBoostingRegressor()
parameters = {'learning_rate':[0.001, 0.01, 0.1, 1], 'n_estimators':[10, 100, 1000],
              'min_samples_split':[2, 3, 4], 'min_samples_leaf':[1, 2, 3], 
              'max_depth':[3]}
X = X_train
y = y_train

optimised_model = cv_optimize(model, parameters, X, y)

BEST {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100} 0.8562806227574921 {'mean_fit_time': array([0.0130259 , 0.07756004, 0.70030699, 0.00931592, 0.07136259,
       0.70455198, 0.00936794, 0.07093296, 0.69458718, 0.00929599,
       0.07071786, 0.69141893, 0.00937967, 0.07063546, 0.69874759,
       0.00996394, 0.07295303, 0.68823352, 0.00931988, 0.0715591 ,
       0.68610835, 0.00956078, 0.0712347 , 0.6900372 , 0.00969534,
       0.07248583, 0.69346399, 0.00946159, 0.07100706, 0.70881305,
       0.00960817, 0.07217832, 0.70020671, 0.00961556, 0.07308197,
       0.69561915, 0.00958891, 0.07184963, 0.68866587, 0.01028733,
       0.07271481, 0.6941361 , 0.00936828, 0.07324176, 0.69027319,
       0.00927515, 0.07224674, 0.68319964, 0.00933504, 0.07190452,
       0.68293877, 0.00938931, 0.0705091 , 0.68601041, 0.00945177,
       0.07145486, 0.70402336, 0.00944366, 0.07129507, 0.69639935,
       0.01045666, 0.07138081, 0.6946476 , 0.00

In [0]:
new_score = optimised_model.score(X_test,y_test)
new_score, scores['GradientBoostingRegressor']

(0.9949751342968375, 0.9949502126992082)

In [0]:
Feature_importance = pd.DataFrame(list(zip(X_train.columns, optimised_model.feature_importances_)))
Feature_importance.columns = ['Feature', 'Importance']
px.bar(data_frame=Feature_importance.sort_values(by='Importance', ascending=False).head(10),
       x = 'Feature',
       y = 'Importance')

##Can we predict the price of an existing listing in the future? Let's say listing#109?